In [4]:
import pandas as pd
import pandas as pd
import numpy as np
import os.path as path

def filter_cold_start_articles(df: pd.DataFrame, min_purchases: int = 5) -> pd.DataFrame:
    """
    Remove cold-start articles based on minimum interaction threshold.

    Args:
    - df: DataFrame, must contain column 'article_id'. 
          Typically includes [t_dat, customer_id, article_id, price, sales_channel_id]
    - min_purchases : int, default 5. The minimum number of total purchases an article must have to be retained in the dataset.

    Return:
    - filtered_df : DataFrame containing only the transactions where article_id has appeared at least `min_purchases` times. Index is reset.
    """
    df = df.copy()
    article_counts = df.groupby('article_id')['article_id'].transform('count')

    return df[article_counts >= min_purchases].reset_index(drop=True)



def filter_weeks_length(
    df: pd.DataFrame,
    weeks: int = 24,
    min_purchases: int = 4,
    max_purchase: int = None
) -> pd.DataFrame:
    """
    Filter user transactions based on their last active date, minimum purchase count,
    and optionally cap the number of retained purchases per user to the last `max_purchase` entries.

    Args:
    - df             : DataFrame, 必須包含 [t_dat, customer_id, article_id, price, sales_channel_id]
    - weeks          : int, default 24. 保留從最後一筆交易往回推 weeks 週的資料
    - min_purchases  : int, default 4. 最少交易筆數，否則整個 user 會被過濾掉
    - max_purchase   : int or None. 若不為 None，對於每個 user 最多保留最後 max_purchase 筆交易

    Return:
    - filtered_df : 經過篩選、上限控制後，並依原始順序排序、重設 index 的 DataFrame
    """
    # 保留原始 index 用於最後還原
    df2 = df.copy()
    df2['t_dat'] = pd.to_datetime(df2['t_dat'])
    df2['customer_id'] = df2['customer_id'].astype('category')

    # 1. 計算每位 user 的最後交易日、交易筆數
    last_date  = df2.groupby('customer_id')['t_dat'].transform('max')
    row_count  = df2.groupby('customer_id')['t_dat'].transform('size')
    start_date = last_date - np.timedelta64(weeks*7, 'D')

    # 2. 時間窗 & min_purchases 篩選
    mask = (
        (df2['t_dat'] >= start_date) &
        (df2['t_dat'] <= last_date) &
        (row_count >= min_purchases)
    )
    filtered = df2[mask]

    # 3. 若需上限，僅對超過 max_purchase 的 user 做排序 & 截斷
    if max_purchase is not None:
        # 計算各 user 在時間窗內的交易數
        counts = filtered.groupby('customer_id').size()
        users_over = counts[counts > max_purchase].index

        # 分組處理：超過上限的 user vs 不超過的 user
        df_over = filtered[filtered['customer_id'].isin(users_over)]
        df_rest = filtered[~filtered['customer_id'].isin(users_over)]

        # 僅對超限者做倒序排名並截斷
        df_over = df_over.sort_values(['customer_id','t_dat'], ascending=[True, False])
        df_over['rank'] = df_over.groupby('customer_id').cumcount()
        df_over = df_over[df_over['rank'] < max_purchase].drop(columns='rank')

        # 合併、還原
        filtered = pd.concat([df_over, df_rest], ignore_index=False)

    # 4. 還原成原始順序、重設 index
    filtered = filtered.sort_index().reset_index(drop=True)
    return filtered



In [ ]:
"""Unit Test
def filter_weeks_length(
    df: pd.DataFrame,
    weeks: int = 24,
    min_purchases: int = 4,
    max_purchase: int = None
) -> pd.DataFrame:
    df2 = df.copy()
    df2['t_dat'] = pd.to_datetime(df2['t_dat'])
    df2['customer_id'] = df2['customer_id'].astype('category')
    last_date  = df2.groupby('customer_id')['t_dat'].transform('max')
    row_count  = df2.groupby('customer_id')['t_dat'].transform('size')
    start_date = last_date - np.timedelta64(weeks*7, 'D')
    mask = (
        (df2['t_dat'] >= start_date) &
        (df2['t_dat'] <= last_date) &
        (row_count >= min_purchases)
    )
    filtered = df2[mask]
    if max_purchase is not None:
        counts = filtered.groupby('customer_id').size()
        users_over = counts[counts > max_purchase].index
        df_over = filtered[filtered['customer_id'].isin(users_over)]
        df_rest = filtered[~filtered['customer_id'].isin(users_over)]
        df_over = df_over.sort_values(['customer_id','t_dat'], ascending=[True, False])
        df_over['rank'] = df_over.groupby('customer_id').cumcount()
        df_over = df_over[df_over['rank'] < max_purchase].drop(columns='rank')
        filtered = pd.concat([df_over, df_rest], ignore_index=False)
    return filtered.sort_index().reset_index(drop=True)

today = datetime(2025, 5, 18)
data = []

for i in range(6):
    data.append({
        'customer_id': 'A',
        'article_id': f'art{i}',
        'price': 10+i,
        'sales_channel_id': 1,
        't_dat': today - timedelta(days=7*i)
    })

data.append({
    'customer_id': 'B',
    'article_id': 'b_art',
    'price': 20,
    'sales_channel_id': 1,
    't_dat': today
})

for i in range(2):
    data.append({
        'customer_id': 'C',
        'article_id': f'c_art{i}',
        'price': 30+i,
        'sales_channel_id': 1,
        't_dat': today - timedelta(days=7*i)
    })

df = pd.DataFrame(data)


filtered = filter_weeks_length(df, weeks=4, min_purchases=2, max_purchase=3)


print("Filtered DataFrame:")
print(filtered)

assert set(filtered['customer_id']) == {'A', 'C'}, "Only users A and C should remain"
assert len(filtered[filtered['customer_id']=='A']) == 3, "User A should have exactly 3 records"
assert len(filtered[filtered['customer_id']=='C']) == 2, "User C should have 2 records"

print("\nAll tests passed!")
"""


Filtered DataFrame:
  customer_id article_id  price  sales_channel_id      t_dat
0           A       art0     10                 1 2025-05-18
1           A       art1     11                 1 2025-05-11
2           A       art2     12                 1 2025-05-04
3           C     c_art0     30                 1 2025-05-18
4           C     c_art1     31                 1 2025-05-11

All tests passed!


C:\Users\louislin\AppData\Local\Temp\ipykernel_48380\3348506245.py:16: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  last_date  = df2.groupby('customer_id')['t_dat'].transform('max')
C:\Users\louislin\AppData\Local\Temp\ipykernel_48380\3348506245.py:17: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  row_count  = df2.groupby('customer_id')['t_dat'].transform('size')
C:\Users\louislin\AppData\Local\Temp\ipykernel_48380\3348506245.py:26: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=T

In [5]:
"""main"""
trans = pd.read_csv(r"transactions_train_mapping.csv",
                 parse_dates=['t_dat'],
                 dtype={
                     'customer_id':'int',
                     'article_id': 'int',
                     'price':'float',
                     'sales_channel_id':'int'
                 })

ARTICLE = path.join("articles_mapping.csv")
article_dtype = {
    'article_id':"int",
    'detail_desc':"category"
}

article = pd.read_csv(ARTICLE,usecols=['article_id','detail_desc'],dtype=article_dtype,engine='pyarrow')

""" Remove the transaction record of Missing desc articles"""
valid_ids = article.loc[article['detail_desc'].notnull(), 'article_id']
trans = trans[trans['article_id'].isin(valid_ids)]
trans.to_csv("transactions_train_mapping_clean.csv",index=False)



In [6]:
# trans_remove_cold.to_csv("transactions_5.csv",index=False)
# trans_minLen_4.to_csv("transactions_5_4.csv",index=False)
# trans_minLen_6.to_csv("transactions_5_6.csv",index=False)
"""Remove Cold Start articles in transactions"""
trans_remove_cold = filter_cold_start_articles(trans,min_purchases = 5)
origin_cust_sum = trans_remove_cold['customer_id'].nunique()

"""Filtering date(last 24 weeks) and session lengths(min = [4,6])"""
trans_minLen_4 = filter_weeks_length(trans_remove_cold, weeks=24, min_purchases = 4, max_purchase= 30)
min4_cust_sum = trans_minLen_4['customer_id'].nunique()

trans_minLen_6 = filter_weeks_length(trans_remove_cold, weeks=24, min_purchases = 6 , max_purchase= 30)
min6_cust_sum = trans_minLen_6['customer_id'].nunique()

print(f"After Filter length 3 : retain ratio ({min4_cust_sum} : {origin_cust_sum}) , remove: {origin_cust_sum-min4_cust_sum}")
print(f"After Filter length 5 : retain ratio ({min6_cust_sum} : {origin_cust_sum}) , remove: {origin_cust_sum-min6_cust_sum}")

trans_remove_cold.to_csv("transactions_5_mapping.csv",index=False)
trans_minLen_4.to_csv("transactions_5_4_30_mapping.csv",index=False)
trans_minLen_6.to_csv("transactions_5_6_30_mapping.csv",index=False)


C:\Users\louislin\AppData\Local\Temp\ipykernel_48380\292377321.py:50: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  last_date  = df2.groupby('customer_id')['t_dat'].transform('max')
C:\Users\louislin\AppData\Local\Temp\ipykernel_48380\292377321.py:51: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  row_count  = df2.groupby('customer_id')['t_dat'].transform('size')
C:\Users\louislin\AppData\Local\Temp\ipykernel_48380\292377321.py:65: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True

After Filter length 3 : retain ratio (1006003 : 1361469) , remove: 355466
After Filter length 5 : retain ratio (859243 : 1361469) , remove: 502226
